In [1]:
using CSV
using DataFrames
using Dates
using Random
using Distributions
using Statistics
using RData
using Base.Threads

In [2]:
function fun_df_possible_si(si_dist, i)
    
    if (nrow(si_dist["si.dist"][i]["si.dist"]) == 0)
        df_possible_si = DataFrame(
            date_infector = Date(si_dist["si.dist"][i]["date"], dateformat"y-m-d"),
            possible_si = missing
        )

    else
        df_possible_si = DataFrame(
            date_infector = Date(si_dist["si.dist"][i]["date"], dateformat"y-m-d"),
            possible_si = Int64.(si_dist["si.dist"][i]["si.dist"].si_week)
        )
    end

    return df_possible_si
end;

In [3]:
function CI_R_case(region::String)
    
    fn = "processed/" * region * "_forward_infector_si.RData"
    
    df_record_full = df[df.region .== region, [:date, :N]];
    rename!(df_record_full, :date => :date_infectee);

    # Period for which we want to estimate R_case
    df_record = df_record_full[df_record_full.date_infectee .>= Date(2009, 1, 1), :];
    
    # load serial interval
    si_dist = load(fn)
    
    df_possible_si = fun_df_possible_si(si_dist, 1);
    for i = 2:length(si_dist["si.dist"])
        df_possible_si = vcat(df_possible_si, fun_df_possible_si(si_dist, i));
    end
    
    # add new column to dataframe
    transform!(df_possible_si, [:date_infector, :possible_si] => ByRow((x, y) -> (ismissing(x) | ismissing(y) ) 
            ? missing : x + Dates.Day(7*y)) => :date_infectee);
    
    df_index_si = DataFrame(
        date_infector = [Date(si_dist["si.dist"][i]["date"], dateformat"y-m-d") for i = 1:length(si_dist["si.dist"])],
        index = 1:length(si_dist["si.dist"])
    );
    
    num_trials = 10000;
    
    # list of infectee symptom onset(or confirmed) date (isod)
    isod = df_record.date_infectee;
    
    R = zeros(num_trials, length(isod));
    
    @time begin
    Threads.@threads for ind in 1:length(isod)
    
        t_j = isod[ind];
        
        possible_t_i = df_possible_si[df_possible_si.date_infector .== t_j, :date_infectee] |> unique |> sort;
        possible_t_i = ((length(possible_t_i) == 0) || first(ismissing.(possible_t_i))) ? possible_t_i : possible_t_i[possible_t_i .< Date(2019, 1, 1)];
        N_at_t_j = df_record[df_record.date_infectee .== t_j, :N];
        
        if ((length(possible_t_i) == 0) || first(ismissing.(possible_t_i))) | first(N_at_t_j .== 0)
            
            R[:, ind] .= 0
        
        else
    
            for ind_3 in 1:N_at_t_j[1]
            
                for ind_2 in 1:length(possible_t_i)
            
                    t_i = possible_t_i[ind_2];
                    
                    # w: w(t_{i} - t_{j}, t_{j})
                    index = df_index_si[df_index_si.date_infector .== t_j, :index];
                    df_tmp = si_dist["si.dist"][first(index)]["si.dist"];
                    w = df_tmp[df_tmp.si_week .== Dates.value(t_i - t_j)/7, :prob];
            
                    possible_t_k = df_possible_si[((.!ismissing.(df_possible_si.date_infectee)) .& 
                            (.!ismissing.(df_possible_si.date_infector))) .& 
                            (df_possible_si.date_infectee .== t_i), :date_infector] |> unique |> sort;
            
                    denom_p_ij = 0;
                    for t_k in possible_t_k
                        index = df_index_si[df_index_si.date_infector .== t_k, :index];
                        df_tmp = si_dist["si.dist"][first(index)]["si.dist"];
                        denom_p_ij = denom_p_ij .+ df_record_full[df_record_full.date_infectee .== t_k, :N] .* 
                            df_tmp[df_tmp.si_week .== Dates.value(t_i - t_k)/7, :prob];
                    end
        
                    R[:, ind] .= R[:, ind] .+ rand(Binomial(df_record_full[df_record_full.date_infectee .== t_i, :N][1], 
                            (w./denom_p_ij)[1]), num_trials);
            
                end
            end
        end
    
        R[:, ind] .= R[:, ind]./df_record_full[df_record_full.date_infectee .== t_j, :N]
        
    end
    end
    
    R_ll = zeros(length(isod));
    R_ll = [length(filter(!isnan, R[:, t])) == 0 ? 0 : quantile(filter(!isnan, R[:, t]), 0.025) for t = 1:length(isod)];
    
    R_ul = zeros(length(isod));
    R_ul = [length(filter(!isnan, R[:, t])) == 0 ? 0 : quantile(filter(!isnan, R[:, t]), 0.975) for t = 1:length(isod)];
    
    R_med = zeros(length(isod));
    R_med = [length(filter(!isnan, R[:, t])) == 0 ? 0 : quantile(filter(!isnan, R[:, t]), 0.5) for t = 1:length(isod)];
    
    R_mean = zeros(length(isod));
    R_mean = [length(filter(!isnan, R[:, t])) == 0 ? 0 : mean(filter(!isnan, R[:, t])) for t = 1:length(isod)];

    result = DataFrame(
                date = df_record.date_infectee,
                region = region,
                R_ll = Float64.(R_ll),
                R_med = Float64.(R_med),
                R_mean = Float64.(R_mean),
                R_ul = Float64.(R_ul)
            )
    
    return result

end;

In [4]:
# load malaria time series
df = DataFrame(CSV.File("input/malaria_kdca.csv"));

# define null dataframe for results
R_case = DataFrame(
    date = Date[], region = String[], R_ll = Float64[], R_med = Float64[], R_mean = Float64[], R_ul = Float64[]
    );

for region in ["Gyeonggi", "Incheon"]
    R_case = vcat(R_case, CI_R_case(region))
end

# write csv
dn = "processed/";
!isdir(dn) ? mkpath(dn) : nothing 
CSV.write(dn * "CI_R_case.csv", R_case);

┌ Warning: Conversion of RData.RExtPtr to Julia is not implemented
└ @ RData ~/.julia/packages/RData/L5u8v/src/convert.jl:198


 41.640486 seconds (592.94 M allocations: 61.804 GiB, 14.19% gc time, 45.39% compilation time)
 13.970116 seconds (181.23 M allocations: 19.153 GiB, 23.03% gc time)
